In [1]:
import asyncio
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

import pandas as pd

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [2]:
from core.services.larp_client import LarpClient
from geckoterminal_py import GeckoTerminalAsyncClient
import pandas as pd
import asyncio


client = LarpClient()
gt = GeckoTerminalAsyncClient()

In [3]:
# Criteria
MIN_POOL_AGE_DAYS = 7
MAX_POOL_AGE_DAYS = 720
MIN_MARKET_CAP = 1_000_000
MAX_MARKET_CAP = 100_000_000
MIN_VOLUME_24H = 100_000
MIN_LIQUIDITY = 10_000
NETWORK = "solana"
DEX = "orca"
QUOTE_ASSET = "SOL"

In [4]:
pools = await gt.get_top_pools_by_network_dex(NETWORK, DEX)

In [5]:
pools["market_cap_usd"] = pd.to_numeric(pools["market_cap_usd"])
pools["volume_usd_h24"] = pd.to_numeric(pools["volume_usd_h24"])
pools["reserve_in_usd"] = pd.to_numeric(pools["reserve_in_usd"])
pools["pool_created_at"] = pd.to_datetime(pools["pool_created_at"]).dt.tz_localize(None)
pools["base"] = pools["name"].apply(lambda x: x.split("/")[0].strip())
pools["quote"] = pools["name"].apply(lambda x: x.split("/")[1].strip())
pools["volume_liquidity_ratio"] = pools["volume_usd_h24"] / pools["reserve_in_usd"]

In [6]:
pools

,id,type,name,base_token_price_usd,base_token_price_native_currency,quote_token_price_usd,quote_token_price_native_currency,address,reserve_in_usd,pool_created_at,...,transactions_h1_sells,transactions_h24_buys,transactions_h24_sells,volume_usd_h24,dex_id,base_token_id,quote_token_id,base,quote,volume_liquidity_ratio
0,solana_AHTTzwf3GmVMJdxWM8v2MSxyjZj8rQR6hyAC3g9...,pool,POPCAT / SOL,1.50779872193283279081341366369096223260383407...,0.010286465277830898272964049,146.739111259193053479398442339674264916237291...,1.0,AHTTzwf3GmVMJdxWM8v2MSxyjZj8rQR6hyAC3g9477Yj,2.584788e+06,2024-07-03 22:39:49,...,788,23160,23504,4.087373e+07,orca,solana_7GCihgDB8fe6KNjn2MYtkzZcRjQy3t9GHdC8uHY...,solana_So1111111111111111111111111111111111111...,POPCAT,SOL,15.813182
1,solana_D6NdKrKNQPmRZCCnG1GqXtF7MMoHB7qR6GU5TkG...,pool,$WIF / SOL,2.71737609637323098940899197183475680787532639...,0.01854722004636222558052729,146.739111259193053479398442339674264916237291...,1.0,D6NdKrKNQPmRZCCnG1GqXtF7MMoHB7qR6GU5TkG59Qz1,2.224150e+06,2024-05-18 20:46:00,...,512,15152,14517,2.656913e+07,orca,solana_EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM6...,solana_So1111111111111111111111111111111111111...,$WIF,SOL,11.945747
2,solana_83v8iPyZihDEjDdY8RdZddyZNyUtXngz69Lgo9K...,pool,SOL / USDC,146.739111259193053479398442339674264916237291...,1.0,0.99977485887035993376012983162542688799613181...,0.00681201505896137,83v8iPyZihDEjDdY8RdZddyZNyUtXngz69Lgo9Kt5d6d,1.126179e+05,2023-07-05 14:50:59,...,613,14340,13768,7.170053e+05,orca,solana_So1111111111111111111111111111111111111...,solana_EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwy...,SOL,USDC,6.366708
3,solana_FpCMFDFGYotvufJ7HrFHsWEiiQCGbkLCtwHiDnh...,pool,SOL / USDC,146.739111259193053479398442339674264916237291...,1.0,1.00015977616880897347895742100855591745381961...,0.00681201505896137,FpCMFDFGYotvufJ7HrFHsWEiiQCGbkLCtwHiDnh7o28Q,6.647568e+06,2023-07-05 14:39:00,...,489,13946,13923,1.554340e+07,orca,solana_So1111111111111111111111111111111111111...,solana_EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwy...,SOL,USDC,2.338209
4,solana_CwZbEdMZdxjnPLcRGRz8PwuvA4tK4iBmS9YZrMv...,pool,bSOL / mSOL,170.248662514432942243074345035730344634972670...,1.16021325912022,179.678665026005927916093626957434076799432913...,1.22447698833769,CwZbEdMZdxjnPLcRGRz8PwuvA4tK4iBmS9YZrMvnrNJr,1.721491e+06,2023-07-05 14:44:01,...,638,12328,12653,1.127439e+06,orca,solana_bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3...,solana_mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJ...,bSOL,mSOL,0.654920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,solana_EBDbBce5YUsgJEP8QNza5wHeFScuW4sYiwhBY8Z...,pool,ATLAS / USDC,0.00309523675739812715655289875273332563324889...,0.00002109231421777985887303513350302600072514,0.99969290187282750308143599349317859340944623...,0.00681304003630862,EBDbBce5YUsgJEP8QNza5wHeFScuW4sYiwhBY8ZV8ZwY,1.476120e+04,2023-07-05 14:39:01,...,31,109,322,1.789540e+03,orca,solana_ATLASXmbPQxBUYbxPsV97usA3fPQYEqzQBUHgiF...,solana_EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwy...,ATLAS,USDC,0.121233
196,solana_5r2SKyVDAmaj8HJak4qRS2oLCn2caf3jmrJi8ik...,pool,Bonk / SOL,0.00002185503218536993012150131518778164244543...,0.000000148797436817439,146.726005418212675936683979623555148389668307...,1.0,5r2SKyVDAmaj8HJak4qRS2oLCn2caf3jmrJi8ikRKj6s,2.161138e+03,2024-02-05 07:53:03,...,12,210,218,7.848424e+02,orca,solana_DezXAZ8z7PnrnRJjz3wXBoRgixCa6xjnB7YaB1p...,solana_So1111111111111111111111111111111111111...,Bonk,SOL,0.363162
197,solana_Ge3RzCy6gyW2iGRDa2w7gTcJvRkSy1RBR8HvvUr...,pool,$WIF / $MYRO,2.72223490189142344042575111758338444421493056...,0.018553154715676275747514165,0.11477057846769320977471984593336726344220004...,0.00078220887464167344663646267994077350814716,Ge3RzCy6gyW2iGRDa2w7gTcJvRkSy1RBR8HvvUrRVnHQ,9.537817e+03,2024-03-23 06:46:00,...,0,195,230,2.287560e+03,orca,solana_EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM6...,solana_HhJpBhRRn4g56VsyLuT8DL5Bv31HkXqsrahTTUC...,$WIF,$MYRO,0.239841
198,solana_J7caZ14NDYVUgKboEqfxH9qyynWbkY7U6u72twV

In [7]:
from datetime import datetime, timedelta

min_pool_date_created = datetime.utcnow() - timedelta(days=MIN_POOL_AGE_DAYS)
max_pool_date_created = datetime.utcnow() - timedelta(days=MAX_POOL_AGE_DAYS)
pools_filtered = pools[(pools["market_cap_usd"] > MIN_MARKET_CAP) & (pools["market_cap_usd"] < MAX_MARKET_CAP) & (pools["volume_usd_h24"] > MIN_VOLUME_24H) & (pools["reserve_in_usd"] > MIN_LIQUIDITY) & (pools["pool_created_at"] < min_pool_date_created) & (pools["pool_created_at"] > max_pool_date_created) & (pools["quote"] == QUOTE_ASSET)]
fee_rates = []
tick_spacings = []
liquidity = []
for i, pool in pools_filtered.iterrows():
    try:
        pool_info = await client.get_pool_info(pool["address"])
        fee_rates.append(pool_info["feeRateBps"])
        tick_spacings.append(pool_info["tickSpacing"])
        liquidity.append(pool_info["liquidity"])
        await asyncio.sleep(2)
    except:
        fee_rates.append(None)
        tick_spacings.append(None)
        liquidity.append(None)
        print(f"Error with pool {pool['address']}")
pools_filtered["fee_rate"] = fee_rates
pools_filtered["tick_spacing"] = tick_spacings
pools_filtered["liquidity"] = liquidity

Error: 500 - {"statusCode":500,"error":"Internal Server Error","message":"An error occurred while fetching pool info"}
Error with pool 8kVPz3px43yiCysesi91jGpEtzjc5AzNATRPQevhuB8p
Error: 500 - {"statusCode":500,"error":"Internal Server Error","message":"An error occurred while fetching pool info"}
Error with pool 8JptKdcwKdARFFfAy4SJtx9j1fMoFyYnGPh4LGqG2pRL
Error: 500 - {"statusCode":500,"error":"Internal Server Error","message":"An error occurred while fetching pool info"}
Error with pool 2TmgxdB26y653ty5P9bmaMbp2axqwfbdjV1ii3pcvC29


In [8]:
pools_filtered.sort_values("volume_liquidity_ratio", ascending=False, inplace=True)
pools_filtered

,id,type,name,base_token_price_usd,base_token_price_native_currency,quote_token_price_usd,quote_token_price_native_currency,address,reserve_in_usd,pool_created_at,...,volume_usd_h24,dex_id,base_token_id,quote_token_id,base,quote,volume_liquidity_ratio,fee_rate,tick_spacing,liquidity
119,solana_9qyqY3BPfc2Ex9BktiZhDJwfqVHChHXwTQpFNH1...,pool,GINNAN / SOL,0.00000201907665288414305644277112080465655945...,0.00000001397897482273685894123435307290850769...,146.725566396059365079484970097803677653801263...,1.0,9qyqY3BPfc2Ex9BktiZhDJwfqVHChHXwTQpFNH136psy,4.896574e+04,2024-07-29 07:26:00,...,1.706224e+05,orca,solana_GinNabffZL4fUj9Vactxha74GDAW8kDPGaHqMtM...,solana_So1111111111111111111111111111111111111...,GINNAN,SOL,3.484526,100.0,128.0,182012584168488
99,solana_8kVPz3px43yiCysesi91jGpEtzjc5AzNATRPQev...,pool,alpha / SOL,0.00784715344053132344724145065400961706309723...,0.000053606106370048034825891,146.732280530450560626080011116018309037105542...,1.0,8kVPz3px43yiCysesi91jGpEtzjc5AzNATRPQevhuB8p,5.443087e+04,2024-09-22 23:22:28,...,1.733381e+05,orca,solana_2zrH2jE542mzB4HABgBjdWMQPtNC5H12pwo1iLp...,solana_So1111111111111111111111111111111111111...,alpha,SOL,3.184555,NaN,NaN,None
13,solana_6P1JbEwYEaXZBP9skhXpZZaaHqbXvd8VwdqNUsk...,pool,ZYN / SOL,0.09129610520601853459086206860103563835262490...,0.000626204201716053,146.739111259193053479398442339674264916237291...,1.0,6P1JbEwYEaXZBP9skhXpZZaaHqbXvd8VwdqNUsk9QkPU,7.973046e+05,2024-03-11 13:23:03,...,2.107711e+06,orca,solana_PzuaVAUH2tfxGZcbBR6kMxeJsBngnsPLFotGJNC...,solana_So1111111111111111111111111111111111111...,ZYN,SOL,2.643546,30.0,64.0,50987667210894
117,solana_8JptKdcwKdARFFfAy4SJtx9j1fMoFyYnGPh4LGq...,pool,🐕 / SOL,0.00889117693625007110685583278777302083371626...,0.000059788023089860257204269,146.648456580026077921291134202224812329421764...,1.0,8JptKdcwKdARFFfAy4SJtx9j1fMoFyYnGPh4LGqG2pRL,4.512132e+04,2024-10-02 03:28:01,...,1.157516e+05,orca,solana_DLScRnWofxiYGqnvZWGy9Gt98MPqKdznaK4TRuk...,solana_So1111111111111111111111111111111111111...,🐕,SOL,2.565341,NaN,NaN,None
181,solana_HU4mukYu86NQBg5XW3FwVUu3Y5oFResowkeUryi...,pool,LOCKIN / SOL,0.05201197847995445653981654850727835935421211...,0.000352654636578887232381281,146.739111259193053479398442339674264916237291...,1.0,HU4mukYu86NQBg5XW3FwVUu3Y5oFResowkeUryi8D6eN,7.721142e+04,2024-06-03 03:58:28,...,1.794425e+05,orca,solana_8Ki8DpuWNxu9VsS3kQbarsCWMcFGWkzzA8pUPto...,solana_So1111111111111111111111111111111111111...,LOCKIN,SOL,2.324041,100.0,128.0,67455201365791
111,solana_9BTidgAt1T18SSr1x9Xetig3gLCE7D1AMyssXMK...,pool,PENG / SOL,0.20708209453997984526587778824359333362437546...,0.001441236814483954133129748,146.689751553117249617468185513180043171872754...,1.0,9BTidgAt1T18SSr1x9Xetig3gLCE7D1AMyssXMKJCLKh,6.759658e+04,2024-03-13 10:03:03,...,1.525771e+05,orca,solana_A3eME5CetyZPBoWbRUwY3tSe25S6tb18ba9ZPbW...,solana_So1111111111111111111111111111111111111...,PENG,SOL,2.257171,100.0,128.0,444721099312
44,solana_8S24t7whcEUn8VuBRXwritKRy27NfdSWLrRz5gw...,pool,MOBILE / SOL,0.00097378354785851562424421539362921179373073...,0.00000657673097721939,146.739111259193053479398442339674264916237291...,1.0,8S24t7whcEUn8VuBRXwritKRy27NfdSWLrRz5gwQXJ9f,1.089048e+05,2024-02-13 04:33:03,...,2.253199e+05,orca,solana_mb1eu7TzEc71KxDpsmsKoucSSuuoGLv1drys1oP...,solana_So1111111111111111111111111111111111111...,MOBILE,SOL,2.068963,16.0,16.0,45225947384185
138,solana_AGsEkdwzLMDdTzjnHPv2QdKbXhcUfG9udDdauWH...,pool,SIGMA / SOL,0.08416929297022537107226046030097955762608311...,0.000594766227056278337851095,146.739111259193053479398442339674264916237291...,1.0,AGsEkdwzLMDdTzjnHPv2QdKbXhcUfG9udDdauWH1E7kk,2.999367e+05,2024-07-12 20:52:59,...,5.606902e+05,orca,solana_5SVG3T9CNQsm2kEwzbRq6hASqh1oGfjqTtLXYUi...,solana_So1111111111111111111111111111111111111...,SIGMA,SOL,1.869362,200.0,256.0,7386736183406
25,solana_7BZzoP3QB2zK3R7WqRzjS5fpeeErgdy3HGzxXrN...,pool,WEN / SOL,0.00010525310070475697426025234324636951763306...,0.000000707592621294964,146.725

In [77]:
from core.data_structures.candles import Candles


async def get_candles(pool, interval):
    trading_pair = f"{pool['base']}/{pool['quote']}"
    address = pool["address"]
    ohlc = await gt.get_ohlcv(NETWORK, address, interval, currency="token")
    ohlc.index = pd.to_datetime(ohlc["timestamp"], unit="s")
    return Candles(candles_df=ohlc, connector_name="orca", trading_pair=trading_pair, interval=interval)

def add_position_metrics(candle, lookback_periods):
    df = candle.data
    results = []
    
    for period in lookback_periods:
        df_period = df.iloc[-period:]
        max_price = df_period['high'].max()
        min_price = df_period['low'].min()
        range_price = max_price - min_price
        range_price_pct = (max_price - df_period['close'].iloc[-1]) / df_period['close'].iloc[-1]
        current_position = (max_price - df_period['close'].iloc[-1]) / range_price if range_price != 0 else 0
        
        results.append({
            'period': period,
            'max_price': max_price,
            'min_price': min_price,
            'current_position': current_position,
            'range_price_pct': range_price_pct,
            'current_price': df_period['close'].iloc[-1]
        })
    
    return results

In [78]:
pools_data = {}
interval = "1d"

for i, pool in pools_filtered.iterrows():
    candle = await get_candles(pool, interval)
    pools_data[pool["address"]] = {
        "candle": candle,
        "metrics": add_position_metrics(candle, [len(candle.data)])[0],
        "pool": pool
    }

In [79]:
# SORT BY CURRENT POSITION
pools_data = dict(sorted(pools_data.items(), key=lambda item: item[1]['metrics']['current_position'], reverse=True))


In [80]:
potential_positions = []

for pool_id, pool_data in pools_data.items():
    candle = pool_data["candle"]
    metrics = pool_data["metrics"]
    fig = candle.fig()
    period = metrics['period']
    max_price = metrics['max_price']
    min_price = metrics['min_price']
    current_position = metrics['current_position']
    current_price = metrics['current_price']
    tick_spacing = pool_data["pool"]["tick_spacing"]
    if current_position < 0.8 or pd.isna(tick_spacing):
        continue
    potential_positions.append({
        "base_symbol": pool_data["pool"]["base"],
        "quote_symbol": pool_data["pool"]["quote"],
        "tick_spacing": int(tick_spacing),
        "lower_price": str(min_price),
        "upper_price": str(current_price),
        "number_of_positions": 1,
    })
    # Add horizontal lines for max price, min price, and current position
    fig.add_hline(y=max_price, line_dash="dash", line_color="green", annotation_text=f"Max Price ({period})", annotation_position="right")
    fig.add_hline(y=min_price, line_dash="dash", line_color="red", annotation_text=f"Min Price ({period})", annotation_position="right")
    fig.add_hline(y=current_price, line_dash="dash", line_color="orange", annotation_text=f"Current Position ({current_position})", annotation_position="right")

    fig.update_layout(title=f"{candle.trading_pair} - Price Range and Current Position",
                      xaxis_title="Date",
                      yaxis_title="Price",
                      showlegend=True)

    fig.show()


In [81]:
potential_positions

[{'base_symbol': 'MOBILE',
  'quote_symbol': 'SOL',
  'tick_spacing': 16,
  'lower_price': '6.18706991514442e-06',
  'upper_price': '6.630856608617993e-06',
  'number_of_positions': 1},
 {'base_symbol': 'BILLY',
  'quote_symbol': 'SOL',
  'tick_spacing': 128,
  'lower_price': '0.000114572451344222',
  'upper_price': '0.0002546411163651981',
  'number_of_positions': 1},
 {'base_symbol': 'GINNAN',
  'quote_symbol': 'SOL',
  'tick_spacing': 128,
  'lower_price': '1.03340670221593e-08',
  'upper_price': '1.475654817086934e-08',
  'number_of_positions': 1},
 {'base_symbol': 'CROWN',
  'quote_symbol': 'SOL',
  'tick_spacing': 64,
  'lower_price': '0.00164128692801621',
  'upper_price': '0.002804966074436767',
  'number_of_positions': 1},
 {'base_symbol': 'PENG',
  'quote_symbol': 'SOL',
  'tick_spacing': 128,
  'lower_price': '0.000675922712216715',
  'upper_price': '0.001474279990032614',
  'number_of_positions': 1},
 {'base_symbol': 'WEN',
  'quote_symbol': 'SOL',
  'tick_spacing': 16,
  '

In [47]:
# position_bundle = await client.create_orca_position_bundle()

In [82]:
position_bundle

{'signature': '2b425QzrCUnCVRWFP2HuBz1e6DksM7ZAuizWmKmNzqzgLHfjKRm226FX6i2ep2ikcFWh3RDReBp913SQtLQrNtDn',
 'positionBundleMint': '4oo3ZiUTEZ8A5DBFJBp2Co3t5bqXtTfXf8uJFPPT2Qfm',
 'positionBundleAddress': '8REd2q8W3kTKsjZXLX6sktqL9KXbEmfrRfnSU1Sr4SNM'}

In [70]:
pools_filtered

,id,type,name,base_token_price_usd,base_token_price_native_currency,quote_token_price_usd,quote_token_price_native_currency,address,reserve_in_usd,pool_created_at,...,volume_usd_h24,dex_id,base_token_id,quote_token_id,base,quote,volume_liquidity_ratio,fee_rate,tick_spacing,liquidity
119,solana_9qyqY3BPfc2Ex9BktiZhDJwfqVHChHXwTQpFNH1...,pool,GINNAN / SOL,0.00000201907665288414305644277112080465655945...,0.00000001397897482273685894123435307290850769...,146.725566396059365079484970097803677653801263...,1.0,9qyqY3BPfc2Ex9BktiZhDJwfqVHChHXwTQpFNH136psy,4.896574e+04,2024-07-29 07:26:00,...,1.706224e+05,orca,solana_GinNabffZL4fUj9Vactxha74GDAW8kDPGaHqMtM...,solana_So1111111111111111111111111111111111111...,GINNAN,SOL,3.484526,100.0,128.0,182012584168488
99,solana_8kVPz3px43yiCysesi91jGpEtzjc5AzNATRPQev...,pool,alpha / SOL,0.00784715344053132344724145065400961706309723...,0.000053606106370048034825891,146.732280530450560626080011116018309037105542...,1.0,8kVPz3px43yiCysesi91jGpEtzjc5AzNATRPQevhuB8p,5.443087e+04,2024-09-22 23:22:28,...,1.733381e+05,orca,solana_2zrH2jE542mzB4HABgBjdWMQPtNC5H12pwo1iLp...,solana_So1111111111111111111111111111111111111...,alpha,SOL,3.184555,NaN,NaN,None
13,solana_6P1JbEwYEaXZBP9skhXpZZaaHqbXvd8VwdqNUsk...,pool,ZYN / SOL,0.09129610520601853459086206860103563835262490...,0.000626204201716053,146.739111259193053479398442339674264916237291...,1.0,6P1JbEwYEaXZBP9skhXpZZaaHqbXvd8VwdqNUsk9QkPU,7.973046e+05,2024-03-11 13:23:03,...,2.107711e+06,orca,solana_PzuaVAUH2tfxGZcbBR6kMxeJsBngnsPLFotGJNC...,solana_So1111111111111111111111111111111111111...,ZYN,SOL,2.643546,30.0,64.0,50987667210894
117,solana_8JptKdcwKdARFFfAy4SJtx9j1fMoFyYnGPh4LGq...,pool,🐕 / SOL,0.00889117693625007110685583278777302083371626...,0.000059788023089860257204269,146.648456580026077921291134202224812329421764...,1.0,8JptKdcwKdARFFfAy4SJtx9j1fMoFyYnGPh4LGqG2pRL,4.512132e+04,2024-10-02 03:28:01,...,1.157516e+05,orca,solana_DLScRnWofxiYGqnvZWGy9Gt98MPqKdznaK4TRuk...,solana_So1111111111111111111111111111111111111...,🐕,SOL,2.565341,NaN,NaN,None
181,solana_HU4mukYu86NQBg5XW3FwVUu3Y5oFResowkeUryi...,pool,LOCKIN / SOL,0.05201197847995445653981654850727835935421211...,0.000352654636578887232381281,146.739111259193053479398442339674264916237291...,1.0,HU4mukYu86NQBg5XW3FwVUu3Y5oFResowkeUryi8D6eN,7.721142e+04,2024-06-03 03:58:28,...,1.794425e+05,orca,solana_8Ki8DpuWNxu9VsS3kQbarsCWMcFGWkzzA8pUPto...,solana_So1111111111111111111111111111111111111...,LOCKIN,SOL,2.324041,100.0,128.0,67455201365791
111,solana_9BTidgAt1T18SSr1x9Xetig3gLCE7D1AMyssXMK...,pool,PENG / SOL,0.20708209453997984526587778824359333362437546...,0.001441236814483954133129748,146.689751553117249617468185513180043171872754...,1.0,9BTidgAt1T18SSr1x9Xetig3gLCE7D1AMyssXMKJCLKh,6.759658e+04,2024-03-13 10:03:03,...,1.525771e+05,orca,solana_A3eME5CetyZPBoWbRUwY3tSe25S6tb18ba9ZPbW...,solana_So1111111111111111111111111111111111111...,PENG,SOL,2.257171,100.0,128.0,444721099312
44,solana_8S24t7whcEUn8VuBRXwritKRy27NfdSWLrRz5gw...,pool,MOBILE / SOL,0.00097378354785851562424421539362921179373073...,0.00000657673097721939,146.739111259193053479398442339674264916237291...,1.0,8S24t7whcEUn8VuBRXwritKRy27NfdSWLrRz5gwQXJ9f,1.089048e+05,2024-02-13 04:33:03,...,2.253199e+05,orca,solana_mb1eu7TzEc71KxDpsmsKoucSSuuoGLv1drys1oP...,solana_So1111111111111111111111111111111111111...,MOBILE,SOL,2.068963,16.0,16.0,45225947384185
138,solana_AGsEkdwzLMDdTzjnHPv2QdKbXhcUfG9udDdauWH...,pool,SIGMA / SOL,0.08416929297022537107226046030097955762608311...,0.000594766227056278337851095,146.739111259193053479398442339674264916237291...,1.0,AGsEkdwzLMDdTzjnHPv2QdKbXhcUfG9udDdauWH1E7kk,2.999367e+05,2024-07-12 20:52:59,...,5.606902e+05,orca,solana_5SVG3T9CNQsm2kEwzbRq6hASqh1oGfjqTtLXYUi...,solana_So1111111111111111111111111111111111111...,SIGMA,SOL,1.869362,200.0,256.0,7386736183406
25,solana_7BZzoP3QB2zK3R7WqRzjS5fpeeErgdy3HGzxXrN...,pool,WEN / SOL,0.00010525310070475697426025234324636951763306...,0.000000707592621294964,146.725

In [84]:
position_to_create = potential_positions[1]
position_to_create["position_bundle_address"] = position_bundle["positionBundleAddress"]
position_to_create

{'base_symbol': 'BILLY',
 'quote_symbol': 'SOL',
 'tick_spacing': 128,
 'lower_price': '0.000114572451344222',
 'upper_price': '0.0002546411163651981',
 'number_of_positions': 1,
 'position_bundle_address': '8REd2q8W3kTKsjZXLX6sktqL9KXbEmfrRfnSU1Sr4SNM'}

In [72]:
await client.open_orca_positions_in_bundle(
    **position_to_create
)

Error: 500 - {"statusCode":500,"error":"Internal Server Error","message":"Unable to fetch Whirlpool at address at JD6sK1G9KKTUw2zzebgN6qL9SrXpJ7x8fuK7UEqh5qY"}


{'error': '{"statusCode":500,"error":"Internal Server Error","message":"Unable to fetch Whirlpool at address at JD6sK1G9KKTUw2zzebgN6qL9SrXpJ7x8fuK7UEqh5qY"}',
 'status': 500}